# RAPIDS: Dask Tutorial

For additional informations:
- https://rapidsai.github.io/projects/cudf/en/0.7.0/dask-xgb-10min.html
-https://dask-ml.readthedocs.io/en/stable/examples/xgboost.html

### Preprocessing

In [0]:
!nvidia-smi

Sat Jul 20 18:58:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# !wget https://github.com/randerzander/notebooks-extended/raw/master/utils/rapids-colab.sh
# !chmod +x rapids-colab.sh
# !./rapids-colab.sh

# import sys, os
# sys.path.append('/usr/local/lib/python3.6/site-packages/')
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-07-20 19:04:02--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 

In [0]:
! pip install dask

     |████████████████████████████████| 768kB 2.8MB/s 


In [0]:
! pip install dask_cuda

     |████████████████████████████████| 532kB 4.1MB/s 
     |████████████████████████████████| 81kB 31.7MB/s 
     |████████████████████████████████| 440kB 52.1MB/s 
     |████████████████████████████████| 491kB 48.0MB/s 
     |████████████████████████████████| 51kB 25.2MB/s 
     |████████████████████████████████| 245kB 50.0MB/s 
     |████████████████████████████████| 276kB 54.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/90/7e/74/bb640d77775e6b6a78bcc3120f9fea4d2a28b2706de1cff37d
  Stored in directory: /root/.cache/pip/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08
  Stored in directory: /root/.cache/pip/wheels/e1/8b/65/3294e5b727440250bda09e8c0153b7ba19d328f661605cb151
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
  Stored in directory: /root/.cache/pip/wheels/40/b9/42/344857b482c954f48bcff6db72d388e30bf2bee4ed14706faa
Successfully built psutil tornado toolz pyyaml heapdict


In [0]:
from dask.distributed import Client, LocalCluster, wait, progress


# we have one GPU, so limit Dask's workers and threads to exactly 1
cluster = LocalCluster(processes=False)
client = Client(cluster)
client

Client Scheduler: inproc://172.28.0.2/138/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.66 GB


In [0]:
! pip install dask_xgboost

     |████████████████████████████████| 142.8MB 221kB/s 
     |████████████████████████████████| 25.2MB 1.3MB/s 


In [0]:
!pip install dask_xgboost
!pip install dask_cudf
!pip install dask_cuml

  Using cached https://files.pythonhosted.org/packages/9f/0a/717a74df509093abffca80bdae0b1f23c8728172598ea406d54a2667aaea/dask_xgboost-0.1.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/5e/45d829bbb3c7489b8b0b77109bbd0b248732973a6b3cb65f54ccaf712c5e/dask-2.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c1/24/5fe7237b2eca13ee0cfb100bec8c23f4e69ce9df852a64b0493d49dae4e0/xgboost-0.90-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b7/14/d3724c89afaf04699acc184c35ae172aa98234920291e529744d66d71733/distributed-2.1.0-py3-none-any.whl
    100% |████████████████████████████████| 25.2MB 2.1MB/s 
  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/64/b4/a904be4184814adb9dfc2e679c4e611392080a32726a657a34cab93b38c2/zict-1.0.0-py2.py3-none-an

  Using cached https://files.pythonhosted.org/packages/38/93/6b884d55e74aba03e81a29c3b681fe1670ad8aabc5f38423bc1dfd75452b/dask_cudf-0.6.1-py3-none-any.whl
    100% |████████████████████████████████| 17.2MB 2.8MB/s 
    100% |████████████████████████████████| 3.2MB 10.8MB/s 
    100% |████████████████████████████████| 163kB 35.6MB/s 
    100% |████████████████████████████████| 235kB 33.5MB/s 
    100% |████████████████████████████████| 512kB 16.5MB/s 
  Running setup.py bdist_wheel for pycparser ... done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
  Found existing installation: numba 0.45.0
    Uninstalling numba-0.45.0:
      Successfully uninstalled numba-0.45.0
  Found existing installation: pycparser 2.18
    Uninstalling pycparser-2.18:
      Successfully uninstalled pycparser-2.18
  Found existing installation: cudf 0.9.0a0+1525.gd3f19ca.dirty
    Uninstalling cudf-0.9.0a0+1525.gd3f19ca.dir

### Testing

#### XGBoost

In [0]:
import dask; print('Dask Version:', dask.__version__)
import dask_cudf; print('Dask cuDF Version:', dask_cudf.__version__)
import dask_xgboost; print('Dask XGBoost Version:', dask_xgboost.__version__)

Dask Version: 1.1.5
Dask cuDF Version: 0.9.0a+1525.gd3f19ca.dirty
Dask XGBoost Version: 0.1.7


In [0]:
import cudf, cuml, xgboost
import dask_cudf, dask_cuml, dask_xgboost

In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [0]:
# Creating a linearly separable dataset using Gaussian Distributions.
# The first half of the number in Y is 0 and the other half 1.
# Therefore I made the first half of the two features quite different from
# the second half of the features (setting the value of the means quite 
# similar) so that make quite simple the classification between the 
# classes (the data is linearly separable).
dataset_len = 200
dlen = int(dataset_len/2)
X_11 = pd.Series(np.random.normal(2,2,dlen))
X_12 = pd.Series(np.random.normal(9,2,dlen))
X_1 = pd.concat([X_11, X_12]).reset_index(drop=True)
X_21 = pd.Series(np.random.normal(1,3,dlen))
X_22 = pd.Series(np.random.normal(7,3,dlen))
X_2 = pd.concat([X_21, X_22]).reset_index(drop=True)
X_31 = pd.Series(np.random.normal(3,1,dlen))
X_32 = pd.Series(np.random.normal(3,4,dlen))
X_3 = pd.concat([X_31, X_32]).reset_index(drop=True)
Y = pd.Series(np.repeat([0,1],dlen))
df = pd.concat([X_1, X_2, X_3, Y], axis=1)
df.columns = ['X1', 'X2', 'X3', 'Y']
df.head()

,X1,X2,X3,Y
0,5.783483,4.631564,3.524137,0
1,4.064722,-0.216973,2.220087,0
2,-1.321688,5.912564,0.852416,0
3,4.689422,2.640566,2.911503,0
4,0.043322,-4.667435,2.487657,0


In [0]:
X = df.drop(['Y'], axis = 1) #.values
y = df['Y']

In [0]:
def preproces(df, X, y, train_size = 0.80):
  # label_encoder object knows how to understand word labels. 
  label_encoder = preprocessing.LabelEncoder() 

  # Encode labels
  y = label_encoder.fit_transform(y) 

  # identify shape and indices
  num_rows, num_columns = df.shape
  delim_index = int(num_rows * train_size)

  # Splitting the dataset in training and test sets
  X_train, y_train = X[:delim_index, :], y[:delim_index]
  X_test, y_test = X[delim_index:, :], y[delim_index:]

  # Checking sets dimensions
  print('X_train dimensions: ', X_train.shape, 'y_train: ', y_train.shape)
  print('X_test dimensions:', X_test.shape, 'y_validation: ', y_test.shape)

  # Checking dimensions in percentages
  total = X_train.shape[0] + X_test.shape[0]
  print('X_train Percentage:', (X_train.shape[0]/total)*100, '%')
  print('X_test Percentage:', (X_test.shape[0]/total)*100, '%')
  
  return X_train, y_train, X_test, y_test

In [0]:
X_train, y_train, X_test, y_test = preproces(df, X, y)

X_train dimensions:  (160, 3) y_train:  (160,)
X_test dimensions: (40, 3) y_validation:  (40,)
X_train Percentage: 80.0 %
X_test Percentage: 20.0 %


In [0]:
# # create Pandas DataFrames for X_train and X_validation
n_columns = X_train.shape[1]
X_train = pd.DataFrame(X_train)
X_train.columns = ['feature_' + str(i) for i in range(n_columns)]
X_test = pd.DataFrame(X_test)
X_test.columns = ['feature_' + str(i) for i in range(n_columns)]

# create Pandas DataFrames for y_train and y_validation
y_train = pd.DataFrame(y_train)
y_train.columns = ['y']
y_test = pd.DataFrame(y_test)
y_test.columns = ['y']

print(X_train.shape)
print(y_train.shape)

(160, 3)
(160, 1)


In [0]:
# Dask settings
npartitions = 1

# create Dask DataFrames for X_train and X_validation
X_train_dask_pdf = dask.dataframe.from_pandas(X_train, npartitions=npartitions)
X_validation_dask_pdf = dask.dataframe.from_pandas(X_test, npartitions=npartitions)

# create Dask cuDF DataFrames for X_train and X_validation
X_train_dask_cudf = dask_cudf.from_dask_dataframe(X_train_dask_pdf)
X_validation_dask_cudf = dask_cudf.from_dask_dataframe(X_validation_dask_pdf)

# create Dask DataFrames for y_train and y_validation
y_train_dask_pdf = dask.dataframe.from_pandas(y_train, npartitions=npartitions)
y_validation_dask_pdf = dask.dataframe.from_pandas(y_test, npartitions=npartitions)

# create Dask cuDF DataFrames for y_train and y_validation
y_train_dask_cudf = dask_cudf.from_dask_dataframe(y_train_dask_pdf)
y_validation_dask_cudf = dask_cudf.from_dask_dataframe(y_validation_dask_pdf)

print(X_train_dask_cudf.shape)
print(y_train_dask_cudf.shape)

(Delayed('int-64deaf78-56e1-478f-81d7-baaaab4f8982'), 3)
(Delayed('int-de4d25b9-8bbb-4025-9198-5ab782a45c6f'), 1)


In [0]:
# # Optional: persist training and validation data into memory
X_train_dask_cudf = X_train_dask_cudf.persist()
X_validation_dask_cudf = X_validation_dask_cudf.persist()
y_train_dask_cudf = y_train_dask_cudf.persist()
y_validation_dask_cudf = y_validation_dask_cudf.persist()
print(X_train_dask_cudf.shape)
print(y_train_dask_cudf.shape)

(Delayed('int-d465e11c-8d74-4f82-acb7-972c58fc1ad2'), 3)
(Delayed('int-31be8ee8-cb50-4cab-8c36-be39c74b1e5b'), 1)


In [0]:
# %%time

# # instantiate params
# params = {}

# bst = dask_xgboost.train(client, params, X_train_dask_cudf, y_train_dask_cudf, num_boost_round=10)

In [0]:
# y_predictions = dask_xgboost.predict(client, bst, X_validation_dask_cudf)
# y_predictions = dask.dataframe.multi.concat([y_predictions], axis=1)

#### Linear Regression

In [0]:
from math import exp
from dask_cuml.linear_model import LinearRegression as cumlOLS_dask

In [0]:
N = 10000

house_price = [0.27*exp(-0.27*_) for _ in range(0,N)] 
house_price = np.array(house_price,dtype=np.float64)
# Normalize data to prevent inf loss
house_price = pd.Series(house_price/house_price.max(axis=0))

house_age = range(0, N) 
house_age = np.array(house_age,dtype=np.float64)
# Normalize data to prevent inf loss
house_age = pd.Series(house_age/house_age.max(axis=0))

X = pd.concat([house_price, house_age], axis=1)

houses_sold = [0.01*exp(0.015*_) for _ in range(0,N)]
houses_sold = np.array(houses_sold,dtype=np.float64)
# Normalize data to prevent inf loss
y = pd.Series(houses_sold/houses_sold.max(axis=0))

df = pd.concat([house_price, house_age, y], axis=1)
df.columns = ['house_price', 'house_age', 'houses_sold']
df.head()

,house_price,house_age,houses_sold
0,1.000000,0.0000,7.283534e-66
1,0.763379,0.0001,7.393610e-66
2,0.582748,0.0002,7.505350e-66
3,0.444858,0.0003,7.618779e-66
4,0.339596,0.0004,7.733922e-66


In [0]:
# X_cudf = cudf.DataFrame.from_pandas(X)
# y_cudf = np.array(y.as_matrix())
# y_cudf = y_cudf[0]
# y_cudf = cudf.Series(y_cudf)

# workers = client.has_what().keys()

# X_df = dask_cudf.from_cudf(X_cudf, npartitions=len(workers)).persist()
# y_df = dask_cudf.from_cudf(y_cudf, npartitions=len(workers)).persist()

In [0]:
# lr = cumlOLS_dask()

# lr.fit(X_df, y_df)

# ret = lr.predict(X_df)

# error_cuml = mean_squared_error(y, ret.compute().to_array())